In [ ]:
from datetime import datetime, timedelta, time
import random

class Driver:
    def __init__(self, driver_id, work_schedule, break_schedule=None):
        """
        :param driver_id: ID водителя
        :param work_schedule: Расписание работы водителя [(день недели, время начала, время окончания), ...]
        :param break_schedule: Расписание перерывов [(день недели, время начала, время окончания), ...]
        """
        self.driver_id = driver_id
        self.work_schedule = work_schedule
        self.break_schedule = break_schedule or []  # По умолчанию нет перерывов

    def is_available(self, current_time):
        """
        Проверяет, доступен ли водитель в указанный момент времени.
        :param current_time: Текущее время (datetime)
        :return: True, если водитель доступен
        """
        for day, start, end in self.work_schedule:
            if current_time.weekday() == day:
                start_time = datetime.combine(current_time.date(), datetime.strptime(start, "%H:%M").time())
                end_time = datetime.combine(current_time.date(), datetime.strptime(end, "%H:%M").time())
                if start_time <= current_time < end_time:
                    # Проверяем, не находится ли водитель на любом из перерывов
                    for b_day, b_start, b_end in self.break_schedule:
                        if current_time.weekday() == b_day:
                            break_start = datetime.combine(current_time.date(), datetime.strptime(b_start, "%H:%M").time())
                            break_end = datetime.combine(current_time.date(), datetime.strptime(b_end, "%H:%M").time())
                            if break_start <= current_time < break_end:
                                return False
                    return True
        return False

class Bus:
    def __init__(self, bus_id, capacity, driver):
        self.bus_id = bus_id
        self.capacity = capacity
        self.driver = driver
        self.current_passengers = 0
        self.active = True

    def check_driver_availability(self, current_time):
        if self.driver is None:
            self.active = False
            return
        if not self.driver.is_available(current_time):
            self.active = False

class Schedule:
    def __init__(self, start_time, end_time, peak_hours):
        self.start_time = start_time
        self.end_time = end_time
        self.current_time = start_time
        self.time_step = timedelta(minutes=1)
        self.peak_hours = peak_hours
        self.route = []
        self.buses = []
        self.drivers = []
        self.total_passengers = 0
        self.left_passengers = 0
        self.schedule_log = []  # Здесь будем хранить события

    def add_stop(self, stop_name, time_to_next, stop_duration, initial_people):
        self.route.append({
            "stop_name": stop_name,
            "time_to_next": time_to_next,
            "stop_duration": stop_duration,
            "people_waiting": initial_people
        })

    def add_bus(self, bus, start_delay=0, start_stop=0):
        bus.start_time = self.start_time + timedelta(minutes=start_delay)
        bus.start_stop = start_stop
        self.buses.append(bus)

    def add_driver(self, driver):
        """Добавляет водителя в расписание."""
        self.drivers.append(driver)

    def find_next_driver(self, current_time):
        """Находит первого доступного водителя для указанного времени."""
        for driver in self.drivers:
            if driver.is_available(current_time):
                return driver
        return None

    def is_peak_hour(self):
        for start, end in self.peak_hours:
            if start <= self.current_time.hour < end:
                return True
        return False

    def is_night_time(self):
        return self.current_time.hour < 6 or self.current_time.hour >= 22

    def run_simulation(self):
      print(f"Начало симуляции: {self.start_time.strftime('%Y-%m-%d %H:%M')}\n")

      bus_positions = {bus.bus_id: bus.start_stop for bus in self.buses}
      bus_departure_times = {bus.bus_id: bus.start_time for bus in self.buses}

      while self.current_time < self.end_time:
          if self.current_time.time() == time(0, 0):  # Новый день
              print(f"\n=== {self.current_time.strftime('%A, %Y-%m-%d')} ===")

          for stop in self.route:
              new_people = 0
              if self.is_night_time():
                  if random.random() < 0.2:
                      new_people = random.randint(0, 1)
              elif self.is_peak_hour():
                  new_people = random.randint(1, 3)
              else:
                  if random.random() < 0.3:
                      new_people = random.randint(0, 2)

              stop["people_waiting"] += new_people

          for bus in self.buses:
              if not bus.active:
                  # Проверка наличия нового водителя
                  next_driver = self.find_next_driver(self.current_time)
                  if next_driver:
                      bus.driver = next_driver
                      bus.active = True
                      print(f"Автобус {bus.bus_id} снова активен с водителем {next_driver.driver_id}.")
                  continue

              bus.check_driver_availability(self.current_time)
              if not bus.active:
                  print(f"\n--- Время: {self.current_time.strftime('%H:%M')} ---")
                  print(f"Автобус {bus.bus_id} завершил движение, так как водитель {bus.driver.driver_id} завершил смену.")
                  continue

              current_pos = bus_positions[bus.bus_id]
              if self.current_time >= bus_departure_times[bus.bus_id]:
                  stop = self.route[current_pos]
                  stop_name = stop["stop_name"]
                  people_waiting = stop["people_waiting"]

                  # Логируем прибытие
                  self.schedule_log.append({
                      "time": self.current_time.strftime('%H:%M'),
                      "day": self.current_time.strftime('%A'),
                      "stop_name": stop_name,
                      "bus_id": bus.bus_id
                  })

                  print(f"\n[{self.current_time.strftime('%H:%M')}] Автобус {bus.bus_id} прибывает на '{stop_name}' (Ожидающих: {people_waiting})")

                  free_space = bus.capacity - bus.current_passengers
                  passengers_to_board = min(free_space, people_waiting)
                  bus.current_passengers += passengers_to_board
                  stop["people_waiting"] -= passengers_to_board
                  self.total_passengers += passengers_to_board
                  print(f"Подобрано {passengers_to_board} пассажиров. Текущая загрузка: {bus.current_passengers}/{bus.capacity}")

                  passengers_to_disembark = random.randint(bus.current_passengers // 3, bus.current_passengers)
                  bus.current_passengers -= passengers_to_disembark
                  print(f"Высажено {passengers_to_disembark} пассажиров.")

                  stop_duration = timedelta(minutes=stop["stop_duration"])
                  bus_departure_times[bus.bus_id] = self.current_time + stop_duration

                  next_pos = (current_pos + 1) % len(self.route)
                  bus_positions[bus.bus_id] = next_pos

                  time_to_next = stop["time_to_next"]
                  bus_departure_times[bus.bus_id] += timedelta(minutes=time_to_next)

          self.current_time += self.time_step

      for stop in self.route:
          self.left_passengers += stop["people_waiting"]

      print("\nСимуляция завершена.")
      print(f"Перевезено пассажиров: {self.total_passengers}")
      print(f"Осталось на остановках: {self.left_passengers}")


    def print_schedule(self):
      print("\n=== Расписание автобусов ===")
      schedule_by_stop = {}

      for log in self.schedule_log:
          stop_name = log["stop_name"]
          if stop_name not in schedule_by_stop:
              schedule_by_stop[stop_name] = []
          schedule_by_stop[stop_name].append(
              (log["day"], log["time"], f"Автобус {log['bus_id']}")
          )

      for stop_name, records in schedule_by_stop.items():
          print(f"\nОстановка: {stop_name}")
          for day, time, bus_info in sorted(records, key=lambda x: (x[0], x[1])):
              print(f"{day} {time} - {bus_info}")

In [ ]:
# Найти ближайший понедельник к текущей дате
def get_next_monday(start_date=None):
    start_date = start_date or datetime.now()
    days_ahead = 0 if start_date.weekday() == 0 else (7 - start_date.weekday())
    return start_date + timedelta(days=days_ahead)

# Начало симуляции - всегда понедельник
start_date = get_next_monday()
start_time = datetime.combine(start_date, datetime.strptime("08:00", "%H:%M").time())
end_time = start_time + timedelta(days=7)

# Часы пик
peak_hours = [(7, 9), (17, 19)]

# Создаем расписание
schedule = Schedule(start_time=start_time, end_time=end_time, peak_hours=peak_hours)

# Остановки
schedule.add_stop("Stop A", 10, 5, 3)
schedule.add_stop("Stop B", 5, 7, 2)
schedule.add_stop("Stop C", 15, 6, 4)
schedule.add_stop("Stop D", 8, 10, 3)
schedule.add_stop("Stop E", 12, 8, 5)
schedule.add_stop("Stop F", 7, 6, 6)
schedule.add_stop("Stop G", 10, 4, 2)
schedule.add_stop("Stop H", 6, 5, 7)
schedule.add_stop("Stop I", 14, 9, 4)
schedule.add_stop("Stop J", 9, 7, 3)

In [ ]:
# Водители с расписанием работы и перерывами
drivers = [
    Driver(driver_id=1,
           work_schedule=[(0, "09:00", "18:00"), (1, "09:00", "18:00"), (2, "09:00", "18:00"), (3, "09:00", "18:00"), (4, "09:00", "18:00")],
           break_schedule=[(0, "13:00", "14:00"), (1, "13:00", "14:00"), (2, "13:00", "14:00"), (3, "13:00", "14:00"), (4, "13:00", "14:00")]),
    Driver(driver_id=2,
           work_schedule=[(0, "09:00", "18:00"), (1, "09:00", "18:00"), (2, "09:00", "18:00"), (3, "09:00", "18:00"), (4, "09:00", "18:00")],
           break_schedule=[(0, "13:00", "14:00"), (1, "13:00", "14:00"), (2, "13:00", "14:00"), (3, "13:00", "14:00"), (4, "13:00", "14:00")]),
    Driver(driver_id=3,
           work_schedule=[(0, "09:00", "18:00"), (1, "09:00", "18:00"), (2, "09:00", "18:00"), (3, "09:00", "18:00"), (4, "09:00", "18:00")],
           break_schedule=[(0, "13:00", "14:00"), (1, "13:00", "14:00"), (2, "13:00", "14:00"), (3, "13:00", "14:00"), (4, "13:00", "14:00")]),
    Driver(driver_id=4,
           work_schedule=[(0, "09:00", "18:00"), (1, "09:00", "18:00"), (2, "09:00", "18:00"), (3, "09:00", "18:00"), (4, "09:00", "18:00")],
           break_schedule=[(0, "13:00", "14:00"), (1, "13:00", "14:00"), (2, "13:00", "14:00"), (3, "13:00", "14:00"), (4, "13:00", "14:00")]),
    Driver(driver_id=5,
           work_schedule=[(0, "09:00", "18:00"), (1, "09:00", "18:00"), (2, "09:00", "18:00"), (3, "09:00", "18:00"), (4, "09:00", "18:00")],
           break_schedule=[(0, "13:00", "14:00"), (1, "13:00", "14:00"), (2, "13:00", "14:00"), (3, "13:00", "14:00"), (4, "13:00", "14:00")]),
    Driver(driver_id=6,
           work_schedule=[(0, "09:00", "18:00"), (1, "09:00", "18:00"), (2, "09:00", "18:00"), (3, "09:00", "18:00"), (4, "09:00", "18:00")],
           break_schedule=[(0, "13:00", "14:00"), (1, "13:00", "14:00"), (2, "13:00", "14:00"), (3, "13:00", "14:00"), (4, "18:00", "18:30")]),
    Driver(driver_id=7,
           work_schedule=[(1, "06:00", "18:00"), (3, "06:00", "18:00"), (5, "06:00", "18:00")],
           break_schedule=[(1, "12:00", "12:20"), (1, "16:00", "16:20"),
                           (3, "12:00", "12:20"), (3, "16:00", "16:20"),
                           (5, "12:00", "12:20"), (5, "16:00", "16:20")]),
    Driver(driver_id=8,
           work_schedule=[(1, "06:00", "18:00"), (3, "06:00", "18:00"), (5, "06:00", "18:00")],
           break_schedule=[(1, "12:00", "12:20"), (1, "16:00", "16:20"),
                           (3, "12:00", "12:20"), (3, "16:00", "16:20"),
                           (5, "12:00", "12:20"), (5, "16:00", "16:20")]),
]

for driver in drivers:
    schedule.add_driver(driver)

# Автобусы
buses = [
    Bus(bus_id=101, capacity=40, driver=drivers[0]),
    Bus(bus_id=102, capacity=50, driver=drivers[1]),
    Bus(bus_id=103, capacity=25, driver=drivers[2]),
    Bus(bus_id=104, capacity=25, driver=drivers[3]),
    Bus(bus_id=105, capacity=25, driver=drivers[4]),
    Bus(bus_id=106, capacity=25, driver=drivers[5]),
    Bus(bus_id=107, capacity=25, driver=drivers[6]),
]
for i, bus in enumerate(buses):
    schedule.add_bus(bus, start_delay=i * 5, start_stop=i)


In [ ]:
# Запуск симуляции
schedule.run_simulation()

Выходные данные были обрезаны до нескольких последних строк (5000).
[14:10] Автобус 107 прибывает на 'Stop G' (Ожидающих: 195)
Подобрано 8 пассажиров. Текущая загрузка: 25/25
Высажено 12 пассажиров.

[14:12] Автобус 102 прибывает на 'Stop B' (Ожидающих: 286)
Подобрано 47 пассажиров. Текущая загрузка: 50/50
Высажено 50 пассажиров.

[14:12] Автобус 105 прибывает на 'Stop E' (Ожидающих: 270)
Подобрано 25 пассажиров. Текущая загрузка: 25/25
Высажено 25 пассажиров.

[14:23] Автобус 106 прибывает на 'Stop G' (Ожидающих: 189)
Подобрано 9 пассажиров. Текущая загрузка: 25/25
Высажено 17 пассажиров.

[14:24] Автобус 102 прибывает на 'Stop C' (Ожидающих: 246)
Подобрано 50 пассажиров. Текущая загрузка: 50/50
Высажено 28 пассажиров.

[14:24] Автобус 107 прибывает на 'Stop H' (Ожидающих: 229)
Подобрано 12 пассажиров. Текущая загрузка: 25/25
Высажено 23 пассажиров.

[14:25] Автобус 101 прибывает на 'Stop B' (Ожидающих: 242)
Подобрано 14 пассажиров. Текущая загрузка: 40/40
Высажено 14 пассажиров.

[14

In [ ]:
schedule.print_schedule()


=== Расписание автобусов ===

Остановка: Stop A
Friday 09:01 - Автобус 101
Friday 09:01 - Автобус 102
Friday 10:18 - Автобус 106
Friday 10:18 - Автобус 107
Friday 10:38 - Автобус 105
Friday 10:56 - Автобус 104
Friday 11:29 - Автобус 103
Friday 11:44 - Автобус 101
Friday 11:44 - Автобус 102
Friday 13:02 - Автобус 106
Friday 13:02 - Автобус 107
Friday 13:21 - Автобус 105
Friday 13:41 - Автобус 104
Friday 14:12 - Автобус 103
Friday 14:27 - Автобус 101
Friday 14:27 - Автобус 102
Friday 15:45 - Автобус 106
Friday 15:45 - Автобус 107
Friday 16:04 - Автобус 105
Friday 16:24 - Автобус 104
Friday 16:55 - Автобус 103
Friday 17:10 - Автобус 101
Friday 17:10 - Автобус 102
Monday 09:01 - Автобус 101
Monday 10:05 - Автобус 107
Monday 10:18 - Автобус 106
Monday 10:38 - Автобус 105
Monday 10:56 - Автобус 104
Monday 11:17 - Автобус 103
Monday 11:29 - Автобус 102
Monday 11:44 - Автобус 101
Monday 12:48 - Автобус 107
Monday 14:01 - Автобус 106
Monday 14:17 - Автобус 105
Monday 14:40 - Автобус 104
Monday

In [ ]:
schedule.left_passengers

21295

In [ ]:
import os
import sys
from contextlib import contextmanager

@contextmanager
def suppress_stdout():
    # Сохраняем оригинальный stdout
    original_stdout = sys.stdout
    # Перенаправляем stdout в os.devnull
    with open(os.devnull, 'w') as devnull:
        sys.stdout = devnull
        try:
            yield
        finally:
            # Восстанавливаем stdout
            sys.stdout = original_stdout


In [ ]:
# Типы водителей
TYPE_1_SCHEDULE = [(0, "09:00", "18:00"), (1, "09:00", "18:00"), (2, "09:00", "18:00"), (3, "09:00", "18:00"), (4, "09:00", "18:00")]
TYPE_1_BREAKS = [(0, "13:00", "14:00"), (1, "13:00", "14:00"), (2, "13:00", "14:00"), (3, "13:00", "14:00"), (4, "13:00", "14:00")]

TYPE_2_SCHEDULE = [(1, "06:00", "18:00"), (3, "06:00", "18:00"), (5, "06:00", "18:00")]
TYPE_2_BREAKS = [(1, "12:00", "12:20"), (1, "16:00", "16:20"),
                 (3, "12:00", "12:20"), (3, "16:00", "16:20"),
                 (5, "12:00", "12:20"), (5, "16:00", "16:20")]

class GeneticAlgorithm:
    def __init__(self, num_generations=50, population_size=20, mutation_rate=0.1):
        self.num_generations = num_generations
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.population = []

    def initialize_population(self):
        # Создаем начальную популяцию
        self.population = []
        for _ in range(self.population_size):
            driver_types = [random.choice([1, 2]) for _ in range(8)]  # 8 водителей с случайными типами
            self.population.append(driver_types)

    def fitness(self, driver_types):
        # Создаем расписание и запускаем симуляцию
        schedule = self.create_new_schedule(driver_types)
        with suppress_stdout():
            schedule.run_simulation()
        # Возвращаем количество оставшихся пассажиров
        return schedule.left_passengers

    def select_parents(self, fitness_scores):
        # Выбираем родителей пропорционально их приспособленности (чем меньше left_passengers, тем лучше)
        total_fitness = sum(1 / (1 + score) for score in fitness_scores)
        probabilities = [(1 / (1 + score)) / total_fitness for score in fitness_scores]
        return random.choices(self.population, probabilities, k=2)

    def crossover(self, parent1, parent2):
        point = random.randint(1, len(parent1) - 1)
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
        return child1, child2

    def mutate(self, individual):
        # С вероятностью mutation_rate изменяем случайного водителя
        if random.random() < self.mutation_rate:
            index = random.randint(0, len(individual) - 1)
            individual[index] = 1 if individual[index] == 2 else 2
        return individual

    def create_new_schedule(self, driver_types):
        # Код создания расписания (из предыдущего ответа)
        start_date = get_next_monday()
        start_time = datetime.combine(start_date, datetime.strptime("08:00", "%H:%M").time())
        end_time = start_time + timedelta(days=7)

        # Часы пик
        peak_hours = [(7, 9), (17, 19)]

        # Создаем расписание
        schedule = Schedule(start_time=start_time, end_time=end_time, peak_hours=peak_hours)

        # Остановки
        schedule.add_stop("Stop A", 10, 5, 3)
        schedule.add_stop("Stop B", 5, 7, 2)
        schedule.add_stop("Stop C", 15, 6, 4)
        schedule.add_stop("Stop D", 8, 10, 3)
        schedule.add_stop("Stop E", 12, 8, 5)
        schedule.add_stop("Stop F", 7, 6, 6)
        schedule.add_stop("Stop G", 10, 4, 2)
        schedule.add_stop("Stop H", 6, 5, 7)
        schedule.add_stop("Stop I", 14, 9, 4)
        schedule.add_stop("Stop J", 9, 7, 3)

        # Создаем водителей
        drivers = []
        for i, driver_type in enumerate(driver_types):
            if driver_type == 1:
                drivers.append(Driver(driver_id=i + 1, work_schedule=TYPE_1_SCHEDULE, break_schedule=TYPE_1_BREAKS))
            elif driver_type == 2:
                drivers.append(Driver(driver_id=i + 1, work_schedule=TYPE_2_SCHEDULE, break_schedule=TYPE_2_BREAKS))

        for driver in drivers:
            schedule.add_driver(driver)

        # Создаем автобусы на основе водителей
        for i, driver in enumerate(drivers):
            bus = Bus(bus_id=100 + i + 1, capacity=40, driver=driver)  # Присваиваем каждому водителю свой автобус
            schedule.add_bus(bus, start_delay=i * 5, start_stop=i % len(schedule.route))  # Позиция по кругу

        return schedule

    def run(self):
        # Инициализация популяции
        self.initialize_population()

        for generation in range(self.num_generations):
            # Оценка приспособленности
            fitness_scores = [self.fitness(individual) for individual in self.population]

            # Отбираем лучших для нового поколения
            new_population = []
            while len(new_population) < self.population_size:
                parent1, parent2 = self.select_parents(fitness_scores)
                child1, child2 = self.crossover(parent1, parent2)
                new_population.append(self.mutate(child1))
                if len(new_population) < self.population_size:
                    new_population.append(self.mutate(child2))

            # Обновляем популяцию
            self.population = new_population

            # Логируем лучшее решение текущего поколения
            best_fitness = min(fitness_scores)
            best_solution = self.population[fitness_scores.index(best_fitness)]
            print(f"Generation {generation + 1}: Best fitness = {best_fitness}, Solution = {best_solution}")

        # Возвращаем лучшее решение
        best_fitness = min(fitness_scores)
        best_solution = self.population[fitness_scores.index(best_fitness)]
        return best_solution

In [ ]:
# Создаем объект генетического алгоритма
ga = GeneticAlgorithm(num_generations=10, population_size=5, mutation_rate=0.1)

# Запускаем оптимизацию
best_solution = ga.run()

print(f"Лучшее решение: {best_solution}")


Generation 1: Best fitness = 11449, Solution = [2, 2, 1, 2, 1, 1, 2, 2]
Generation 2: Best fitness = 11544, Solution = [2, 1, 2, 2, 1, 1, 2, 2]
Generation 3: Best fitness = 11578, Solution = [2, 2, 2, 1, 1, 2, 2, 2]
Generation 4: Best fitness = 11460, Solution = [2, 2, 1, 1, 1, 1, 2, 2]
Generation 5: Best fitness = 11495, Solution = [2, 2, 1, 1, 1, 1, 2, 2]
Generation 6: Best fitness = 11517, Solution = [2, 2, 2, 1, 1, 1, 2, 2]
Generation 7: Best fitness = 11558, Solution = [2, 2, 2, 1, 1, 1, 2, 2]
Generation 8: Best fitness = 11572, Solution = [2, 2, 2, 1, 1, 1, 2, 2]
Generation 9: Best fitness = 11529, Solution = [2, 2, 2, 1, 1, 2, 2, 2]
Generation 10: Best fitness = 11558, Solution = [2, 2, 2, 1, 1, 1, 2, 2]
Лучшее решение: [2, 2, 2, 1, 1, 1, 2, 2]


In [ ]:
schedule.buses